#### 1. Importing libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

#### 1.1 Function definition

In [2]:
# Function to parse the PDB file and extract CA atom coordinates
def parse_pdb(file_path):
    atom_coords = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("ATOM") and line[13:15].strip() == "CA":
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                atom_coords.append((x, y, z))
    return np.array(atom_coords)

In [3]:
def generate_contact_map(atom_coords, distance_threshold):

    num_residues = len(atom_coords)
    contact_map = np.zeros((num_residues, num_residues), dtype=bool)
    
    for i in range(num_residues):
        for j in range(i + 4, num_residues):
            distance = np.linalg.norm(atom_coords[i] - atom_coords[j])
            if distance < distance_threshold:
                contact_map[i, j] = True
                contact_map[j, i] = True
    
    return contact_map

In [4]:
# Function to save the contact map as a .png file
def save_contact_map(contact_map, output_path):
    plt.figure(figsize=(10, 10))
    plt.imshow(contact_map, cmap='Greys', interpolation='none')
    plt.title('Contact Map')
    plt.xlabel('Residue Index')
    plt.ylabel('Residue Index')
    plt.savefig(output_path)
    plt.close()

In [5]:
# Function to process a single PDB file
def process_single_pdb(input_file, output_dir, distance_threshold):
    atom_coords = parse_pdb(input_file)
    contact_map = generate_contact_map(atom_coords, distance_threshold)
    
    if 'alpha-helices' in input_file:
        specific_output_dir = os.path.join(output_dir, 'alpha-helices')
    elif 'beta-sheets' in input_file:
        specific_output_dir = os.path.join(output_dir, 'beta-sheets')
    else:
        specific_output_dir = output_dir
    
    if not os.path.exists(specific_output_dir):
        os.makedirs(specific_output_dir)
    
    base_name = os.path.basename(input_file).replace('_bundle', '')
    output_path = os.path.join(specific_output_dir, f'{base_name[:-4]}.png')
    
    save_contact_map(contact_map, output_path)
    return output_path

In [6]:
# Function to process all PDB files
def process_pdb_files(input_dirs, output_dir, distance_threshold):
    pdb_files = []
    for input_dir in input_dirs:
        for filename in os.listdir(input_dir):
            if filename.endswith('.pdb'):
                file_path = os.path.join(input_dir, filename)
                pdb_files.append(file_path)
    
    for pdb_file in pdb_files:
        process_single_pdb(pdb_file, output_dir, distance_threshold)

#### 1.2 Definition of variables

In [7]:
# Define input directories
input_dirs = ['../data/alpha-helices', '../data/beta-sheets']

# Define output directory
output_dir = '../data/contact-maps'

# Define the distance threshold in angstroms for contact map generation
distance_threshold = 8.0

In [134]:
# Process the PDB files sequentially
process_pdb_files(input_dirs, output_dir, distance_threshold)